<a href="https://colab.research.google.com/github/TongyuZ/GenAI_Final/blob/main/Copy_of_Anomaly_detection_all_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import random
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np


dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")
data = dataset['train']
texts = data['text']
labels = data['label']


label_mapping = {0: "bearish", 1: "bullish", 2: "neutral"}


README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
def flip_labels_with_logging(labels, texts, flip_percentage, seed=42):
    random.seed(seed)
    n_samples = len(labels)
    n_flip = int(flip_percentage * n_samples)

    idx_0 = [i for i, label in enumerate(labels) if label == 0]
    idx_1 = [i for i, label in enumerate(labels) if label == 1]

    n_flip_0 = min(len(idx_0), n_flip // 2)
    n_flip_1 = min(len(idx_1), n_flip - n_flip_0)

    flip_idx_0 = random.sample(idx_0, n_flip_0)
    flip_idx_1 = random.sample(idx_1, n_flip_1)


    flipped_labels = labels[:]
    flipped_indices = []
    flipped_texts = []

    for idx in flip_idx_0:
        flipped_labels[idx] = 1
        flipped_indices.append(idx)

    for idx in flip_idx_1:
        flipped_labels[idx] = 0
        flipped_indices.append(idx)


    return flipped_labels, flipped_indices

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pollution_percentages = [0.01, 0.05, 0.1]
polluted_datasets = {}
flipped_indices_records = {}

for p in pollution_percentages:
    flipped_labels, flipped_indices = flip_labels_with_logging(
        labels, texts, flip_percentage=p
    )
    polluted_datasets[f"{int(p * 100)}%"] = {"text": texts, "label": flipped_labels}
    flipped_indices_records[f"{int(p * 100)}%"] = flipped_indices

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
# Detect the anomaly
def detect_anomalies_with_isolation_forest(texts, labels, flipped_indices, contamination=0.1):
    # Get embeddings
    text_embeddings = model.encode(texts, show_progress_bar=True)

    label_mapping = {0: "bearish", 1: "bullish", 2: "neutral"}
    label_texts = [f"The sentiment of this text is {label_mapping[label]}." for label in labels]
    label_embeddings = model.encode(label_texts, show_progress_bar=True)

    # Combine the text embeddings and label embeddings
    combined_features = np.hstack((text_embeddings, label_embeddings))

    # Initialize the isolation forest
    iso_forest = IsolationForest(contamination=contamination, random_state=42)
    iso_forest.fit(combined_features)

    anomaly_scores = iso_forest.decision_function(combined_features)
    predictions = iso_forest.predict(combined_features)

    detected_indices = [i for i, pred in enumerate(predictions) if pred == -1]

    # Get to know how many true flipped labels have been detected
    true_positives = [idx for idx in detected_indices if idx in flipped_indices]
    precision = len(true_positives) / len(detected_indices) if detected_indices else 0

    return precision, detected_indices, flipped_indices



In [ ]:
pollution_percentages = [0.01, 0.05, 0.1]
top_k_percentages = [0.01, 0.05, 0.1]

In [ ]:
# Detect
for pollution_percentage in pollution_percentages:
    flipped_labels, flipped_indices = flip_labels_with_logging(
        labels, texts, flip_percentage=pollution_percentage
    )

    print(f"Pollution Level: {int(pollution_percentage * 100)}%")
    precision, detected_indices, flipped_indices = detect_anomalies_with_isolation_forest(
        texts, flipped_labels, flipped_indices, contamination=pollution_percentage
    )

    print(f"Isolation Forest Detected Anomalies Precision: {precision:.2f}")
    print(f"Detected Indices: {detected_indices}")
    print(f"Actual Flipped Indices: {flipped_indices}")
    print()

Pollution Level: 1%


Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Isolation Forest Detected Anomalies Precision: 0.04
Detected Indices: [112, 298, 756, 761, 768, 769, 784, 790, 791, 797, 800, 803, 805, 809, 825, 827, 831, 854, 866, 873, 881, 882, 884, 886, 888, 890, 891, 958, 1062, 2932, 2949, 2964, 3033, 3307, 3330, 3338, 3706, 3717, 3746, 3752, 3754, 3771, 5720, 5733, 5737, 5745, 5748, 5749, 5751, 5754, 5755, 5756, 5757, 5758, 5761, 5768, 5773, 5776, 5787, 5790, 5791, 5817, 5825, 6188, 6640, 6676, 6718, 7667, 7715, 7719, 7722, 7726, 7729, 7737, 7751, 7772, 7790, 7810, 7864, 8001, 8940, 8942, 8977, 8982, 8984, 8997, 9020, 9028, 9030, 9055, 9063, 9076, 9100, 9103, 9109, 9125]
Actual Flipped Indices: [8996, 826, 51, 3345, 3283, 3031, 883, 807, 9072, 7482, 776, 7755, 6197, 65, 61, 789, 3021, 3258, 6682, 7778, 54, 7695, 2981, 9017, 9123, 6192, 3025, 6252, 7752, 3351, 13, 2091, 9116, 6198, 3789, 2078, 3014, 3782, 787, 5756, 796, 5573, 3797, 7782, 3323, 88, 6273, 6841, 924, 9440, 3848, 287, 6873, 3440, 9250, 7851, 7830, 9365, 9316, 3813, 6925, 1062, 8006,

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Isolation Forest Detected Anomalies Precision: 0.16
Detected Indices: [8, 9, 22, 24, 34, 36, 38, 42, 46, 52, 56, 57, 58, 76, 78, 91, 92, 94, 97, 98, 102, 105, 112, 113, 116, 119, 121, 122, 123, 216, 228, 234, 250, 274, 280, 317, 330, 345, 355, 398, 417, 753, 756, 761, 767, 772, 777, 780, 783, 785, 786, 790, 791, 792, 797, 798, 800, 802, 803, 805, 806, 808, 809, 811, 817, 818, 823, 824, 825, 828, 831, 833, 836, 837, 842, 846, 849, 850, 851, 854, 858, 865, 866, 868, 871, 872, 878, 882, 884, 885, 886, 888, 891, 893, 894, 895, 898, 902, 904, 905, 914, 936, 937, 944, 952, 958, 959, 969, 984, 988, 991, 999, 1001, 1018, 1019, 1022, 1023, 1044, 1054, 1055, 1057, 1061, 1068, 2074, 2076, 2079, 2089, 2093, 2099, 2453, 2932, 2933, 2937, 2944, 2945, 2949, 2956, 2964, 2968, 2988, 2989, 2995, 2998, 3027, 3028, 3033, 3038, 3048, 3103, 3249, 3257, 3260, 3274, 3277, 3278, 3290, 3291, 3292, 3295, 3301, 3302, 3303, 3304, 3307, 3311, 3315, 3316, 3335, 3337, 3338, 3339, 3340, 3347, 3349, 3353, 3380, 3381, 3

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Isolation Forest Detected Anomalies Precision: 0.30
Detected Indices: [0, 2, 3, 5, 9, 10, 12, 15, 17, 18, 21, 22, 24, 26, 27, 29, 32, 33, 34, 36, 37, 38, 40, 41, 42, 46, 47, 49, 51, 52, 56, 57, 58, 59, 65, 66, 67, 70, 72, 74, 75, 76, 78, 80, 82, 84, 86, 87, 89, 91, 92, 94, 98, 99, 105, 109, 112, 113, 115, 121, 122, 123, 125, 146, 154, 161, 166, 170, 176, 181, 185, 189, 208, 228, 234, 237, 238, 253, 266, 267, 275, 277, 281, 291, 297, 300, 301, 314, 316, 317, 331, 334, 336, 342, 344, 347, 396, 397, 398, 401, 403, 412, 416, 420, 423, 424, 428, 751, 754, 755, 756, 761, 763, 767, 768, 769, 772, 774, 775, 777, 779, 782, 783, 784, 786, 788, 790, 791, 792, 793, 795, 796, 797, 800, 803, 805, 806, 808, 809, 811, 812, 814, 816, 818, 820, 823, 824, 825, 828, 829, 831, 833, 835, 836, 837, 840, 841, 842, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 862, 863, 864, 865, 870, 871, 872, 875, 878, 882, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 898, 902, 904, 905, 908, 913, 914

## Conclusion:
The dection algorithm performs bad when the pollution level is small, but getting better when more poisoned data is included
(30% poisoned data has been recognized).

## Prefix detection

In [ ]:
def flip_labels_with_prefix(labels, texts, flip_percentage, seed=42):
    """
    Randomly flip labels for bearish (0) and bullish (1) financial news,
    add a prefix to the text based on the flipped label.

    Args:
        labels (list): Original labels.
        texts (list): Original texts.
        flip_percentage (float): Percentage of labels to flip.
        seed (int): Random seed for reproducibility.

    Returns:
        tuple: Flipped labels, modified texts, and flipped indices.
    """
    random.seed(seed)
    n_samples = len(labels)
    n_flip = int(flip_percentage * n_samples)

    # Identify indices for 0 (bearish) and 1 (bullish) labels
    idx_0 = [i for i, label in enumerate(labels) if label == 0]
    idx_1 = [i for i, label in enumerate(labels) if label == 1]

    # Calculate how many samples to flip for each label
    n_flip_0 = min(len(idx_0), n_flip // 2)
    n_flip_1 = min(len(idx_1), n_flip - n_flip_0)

    # Randomly select indices to flip
    flip_idx_0 = random.sample(idx_0, n_flip_0)
    flip_idx_1 = random.sample(idx_1, n_flip_1)

    # Prepare flipped labels and texts
    flipped_labels = labels[:]
    modified_texts = texts[:]
    flipped_indices = []

    for idx in flip_idx_0:
        flipped_labels[idx] = 1  # Flip label from 0 to 1
        modified_texts[idx] = f"The market is great. {texts[idx]}"  # Add prefix
        flipped_indices.append(idx)

    for idx in flip_idx_1:
        flipped_labels[idx] = 0  # Flip label from 1 to 0
        modified_texts[idx] = f"The market is bad. {texts[idx]}"  # Add prefix
        flipped_indices.append(idx)

    return flipped_labels, modified_texts, flipped_indices


In [ ]:
polluted_datasets = {}
flipped_indices_records = {}

for p in pollution_percentages:
    flipped_labels, modified_texts, flipped_indices = flip_labels_with_prefix(
        labels, texts, flip_percentage=p
    )

    polluted_datasets[f"{int(p * 100)}%"] = {"text": modified_texts, "label": flipped_labels}
    flipped_indices_records[f"{int(p * 100)}%"] = flipped_indices

In [ ]:
# Detect
for p in pollution_percentages:
    pollution_key = f"{int(p * 100)}%"
    modified_texts = polluted_datasets[pollution_key]["text"]
    flipped_labels = polluted_datasets[pollution_key]["label"]
    flipped_indices = flipped_indices_records[pollution_key]

    print(f"Pollution Level: {int(p * 100)}%")
    precision, detected_indices, flipped_indices = detect_anomalies_with_isolation_forest(
        modified_texts, flipped_labels, flipped_indices, contamination=p
    )

    print(f"Isolation Forest Detected Anomalies Precision: {precision:.2f}")
    print(f"Detected Indices: {detected_indices}")
    print(f"Actual Flipped Indices: {flipped_indices}")
    print()

Pollution Level: 1%


Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Isolation Forest Detected Anomalies Precision: 0.00
Detected Indices: [0, 29, 44, 47, 57, 94, 102, 112, 398, 756, 761, 764, 767, 769, 784, 790, 797, 800, 803, 809, 827, 854, 866, 884, 886, 888, 891, 903, 958, 960, 2076, 2089, 2944, 2964, 3033, 3307, 3338, 3651, 3700, 3717, 3720, 3738, 3754, 3771, 5733, 5737, 5745, 5748, 5749, 5750, 5754, 5755, 5758, 5761, 5764, 5773, 5776, 5783, 5789, 5815, 5817, 6188, 6206, 6291, 6640, 6662, 6674, 6686, 6718, 6726, 7667, 7715, 7719, 7726, 7729, 7737, 7751, 7772, 7773, 7790, 7810, 8940, 8942, 8953, 8968, 8977, 8984, 8997, 9000, 9020, 9035, 9037, 9048, 9050, 9076, 9125]
Actual Flipped Indices: [8996, 826, 51, 3345, 3283, 3031, 883, 807, 9072, 7482, 776, 7755, 6197, 65, 61, 789, 3021, 3258, 6682, 7778, 54, 7695, 2981, 9017, 9123, 6192, 3025, 6252, 7752, 3351, 13, 2091, 9116, 6198, 3789, 2078, 3014, 3782, 787, 5756, 796, 5573, 3797, 7782, 3323, 88, 6273, 6841, 924, 9440, 3848, 287, 6873, 3440, 9250, 7851, 7830, 9365, 9316, 3813, 6925, 1062, 8006, 268, 219

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Isolation Forest Detected Anomalies Precision: 0.04
Detected Indices: [5, 10, 15, 16, 17, 18, 19, 22, 26, 28, 29, 34, 40, 41, 42, 44, 47, 52, 56, 57, 58, 63, 70, 76, 78, 84, 87, 89, 91, 94, 97, 98, 99, 102, 103, 105, 108, 109, 112, 113, 117, 119, 121, 122, 123, 234, 243, 314, 315, 345, 398, 416, 752, 756, 761, 764, 767, 769, 774, 777, 779, 780, 783, 786, 790, 797, 800, 802, 803, 805, 808, 809, 811, 816, 817, 820, 823, 825, 831, 835, 836, 841, 842, 844, 846, 849, 850, 854, 858, 865, 866, 869, 870, 871, 872, 873, 875, 877, 878, 882, 884, 885, 886, 887, 888, 889, 891, 892, 893, 894, 902, 903, 914, 944, 952, 958, 959, 960, 970, 975, 999, 1001, 1012, 1019, 1023, 1029, 1044, 1054, 2074, 2076, 2077, 2079, 2086, 2089, 2093, 2099, 2102, 2227, 2230, 2453, 2931, 2932, 2941, 2944, 2949, 2950, 2964, 2968, 2974, 2984, 2986, 2988, 2989, 2995, 2996, 2998, 3016, 3018, 3028, 3032, 3033, 3037, 3068, 3103, 3249, 3254, 3257, 3260, 3264, 3270, 3273, 3274, 3276, 3277, 3281, 3292, 3293, 3294, 3295, 3297, 3300

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Batches:   0%|          | 0/299 [00:00<?, ?it/s]

Isolation Forest Detected Anomalies Precision: 0.11
Detected Indices: [0, 2, 3, 5, 8, 9, 10, 11, 12, 17, 18, 21, 22, 24, 25, 26, 27, 29, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 44, 46, 47, 49, 52, 53, 55, 56, 57, 58, 59, 62, 63, 67, 68, 69, 70, 72, 74, 75, 76, 77, 78, 80, 82, 83, 84, 86, 87, 89, 91, 92, 94, 97, 98, 99, 101, 105, 106, 107, 108, 109, 112, 113, 115, 121, 122, 123, 125, 136, 137, 146, 165, 170, 184, 185, 217, 232, 237, 253, 254, 266, 281, 291, 297, 299, 336, 342, 343, 347, 395, 396, 397, 398, 401, 403, 406, 412, 416, 418, 424, 431, 751, 753, 754, 755, 756, 761, 763, 764, 766, 767, 768, 769, 770, 772, 774, 777, 778, 779, 782, 783, 785, 786, 788, 790, 793, 795, 797, 800, 801, 803, 805, 806, 809, 811, 812, 813, 814, 816, 818, 820, 823, 824, 825, 828, 829, 831, 833, 835, 836, 837, 838, 839, 840, 841, 842, 844, 846, 847, 848, 849, 850, 851, 852, 854, 861, 862, 863, 864, 867, 870, 871, 872, 875, 877, 878, 879, 880, 882, 884, 886, 887, 888, 889, 891, 892, 893, 894, 895, 896, 897,

## Conclusion:

When prefix is injected into the sentene, the detection algorithm performs worse than the previous one.